# Task 2: LDA Topic Model
***

We create a simple, out-of-the-box Latent Dirichlet Allocation Topic Model with visualization.

In [21]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import pandas as pd
import numpy as np

from nltk.stem import PorterStemmer

from sklearn.decomposition import LatentDirichletAllocation

from sklearn.feature_extraction.text import CountVectorizer

In [22]:
df = pd.read_csv('cleaned_jels.csv', encoding='utf-8-sig', low_memory=False)
df.drop('0', axis=1, inplace=True)
df

,Title,Publisher,Abstract,Keywords,Jel Codes,Author1,Author2,Author3,Author4,Author5,...,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80
0,technical efficiency of hungarian farms before...,european association of agricultural economists,hungary is one of the ten countries that have ...,"[agribusiness, production economics]",NaN,"zoltán bakucs, lajos","fertő, imre","fogarasi, józsef",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,exploring the flexibility of polish family far...,european association of agricultural economists,no abstract is available for this item,"[farm management, productivity analysis]",NaN,"pieniadz, agata","renner, swetlana","petrick, martin",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,regional clusters in a function of rural devel...,european association of agricultural economists,as a theoretical concept rural development bas...,[community rural urban development],NaN,"sudarić, tihana","zmaić, krunoslav","petrač, božidar",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,the extended metropolitan area in a new member...,european association of agricultural economists,no abstract is available for this item,"[agricultural and food policy, community rural...",NaN,"zolin, m. bruna",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,infrastructural capacity of family owned holdi...,european association of agricultural economists,no abstract is available for this item,"[community rural urban development, land econo...",NaN,"Živković, dragić","dimitrijević, bojan","jelić, sreten","rajić, zoran",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,comparative research of food consumption in se...,european association of agricultural economists,no abstract is available for this item,[food consumption nutrition food safety],NaN,"Мilanović, milan r.","Đorović, milutin","stevanović, simo",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,targeting agricultural and rural development m...,european association of agricultural economists,no abstract is available for this item,"[agricultural and food policy, community rural...",NaN,"segré, andrea","rakić, renata","rokvić, gordana","vittuari, matteo",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,needs assessment analysis of small rural house...,european association of agricultural economists,no abstract is available for this item,[consumer household economics],NaN,"bogdanov, natalija","moslavac, nenad",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,designing a rural development strategy for ser...,european association of agricultural economists,no abstract is available for this item,[community rural urban development],NaN,"cochrane, nancy","reed, michael","jovanović, zlatko",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,rural development and the heritage of chayanov...,european association of agricultural economists,no abstract is available for this item,"[community rural urban development, labor and ...",NaN,"nuppenau, ernst-august",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


>We need to stem all words to reduce dimensionality: jogging, jogged, jog -> jog

In [28]:
stemmer = PorterStemmer()

class Stemmer_CountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(Stemmer_CountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(word) for word in analyzer(doc)])

In [30]:
count_vectorizer = Stemmer_CountVectorizer(stop_words='english', lowercase=True, max_df=.5, min_df=10)
dtm_cv = count_vectorizer.fit_transform(df['Abstract'])

>We use a Latent Dirichlet Allocation Model. 

In [31]:
lda_cv = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_cv.fit(dtm_cv)

/usr/local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_jobs=1, n_topics=20, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [32]:
pyLDAvis.sklearn.prepare(lda_cv, dtm_cv, count_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
18     10.134392        1       1 -0.142399  0.043960
17     10.021352        1       2 -0.161322 -0.041799
0       8.978041        1       3 -0.135488  0.094903
11      8.730728        1       4 -0.151930 -0.082815
7       7.150018        1       5 -0.169611  0.011227
15      6.952235        1       6 -0.118747  0.095571
12      5.573337        1       7 -0.058000  0.251980
14      5.203284        1       8 -0.106242 -0.172855
6       5.071363        1       9 -0.091999 -0.127311
10      4.414702        1      10  0.321169 -0.000713
1       4.263050        1      11 -0.092261  0.091826
13      4.101754        1      12 -0.087417 -0.042403
9       3.920375        1      13 -0.038008 -0.237656
3       3.650140        1      14 -0.040651 -0.154408
4       3.563986        1      15 -0.034538  0.229365
16      2.154483        1      16  0.284727 -0.004624
19      1.733426        1      17  0.110024  0.148310
8       1.589907        1      18  0.246468 -0.017072
2       1.437595        1      19  0.301502 -0.011128
5       1.355832        1      20  0.164721 -0.074358, topic_info=      Category          Freq          Term         Total  loglift  logprob
term                                                                      
1666   Default  26817.000000         avail  26817.000000  30.0000  30.0000
11215  Default  37559.000000         price  37559.000000  29.0000  29.0000
635    Default  25072.000000      abstract  25072.000000  28.0000  28.0000
7906   Default  24911.000000          item  24911.000000  27.0000  27.0000
9297   Default  56946.000000         model  56946.000000  26.0000  26.0000
8212   Default  27810.000000            la  27810.000000  25.0000  25.0000
11313  Default  42814.000000       product  42814.000000  24.0000  24.0000
3560   Default  31583.000000       countri  31583.000000  23.0000  23.0000
8847   Default  44482.000000        market  44482.000000  22.0000  22.0000
3528   Default  24744.000000          cost  24744.000000  21.0000  21.0000
5338   Default  28014.000000         estim  28014.000000  20.0000  20.0000
11716  Default  27933.000000          rate  27933.000000  19.0000  19.0000
10940  Default  37030.000000        polici  37030.000000  18.0000  18.0000
14288  Default  19867.000000         trade  19867.000000  17.0000  17.0000
5709   Default  19246.000000          farm  19246.000000  16.0000  16.0000
886    Default  20194.000000    agricultur  20194.000000  15.0000  15.0000
6007   Default  14571.000000          food  14571.000000  14.0000  14.0000
5904   Default  22076.000000          firm  22076.000000  13.0000  13.0000
14883  Default  59464.000000           use  59464.000000  12.0000  12.0000
1769   Default  18303.000000          bank  18303.000000  11.0000  11.0000
4040   Default   9266.000000           der   9266.000000  10.0000  10.0000
4982   Default  13132.000000            en  13132.000000   9.0000   9.0000
3810   Default  28376.000000          data  28376.000000   8.0000   8.0000
4168   Default  31825.000000       develop  31825.000000   7.0000   7.0000
12835  Default  10064.000000        servic  10064.000000   6.0000   6.0000
7461   Default  15714.000000      industri  15714.000000   5.0000   5.0000
4876   Default  10699.000000            el  10699.000000   4.0000   4.0000
4779   Default  40824.000000        effect  40824.000000   3.0000   3.0000
5875   Default  16546.000000       financi  16546.000000   2.0000   2.0000
13160  Default  13813.000000        social  13813.000000   1.0000   1.0000
...        ...           ...           ...           ...      ...      ...
11551  Topic20    826.421006         puzzl    827.411018   4.2996  -4.7473
8857   Topic20    810.800789       marriag    811.790801   4.2995  -4.7664
6372   Topic20    787.873114         genet    788.863127   4.2995  -4.7951
7045   Topic20    698.855538          host    699.845550   4.2993  -4